In [34]:
# download the search library
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
import minsearch

In [3]:
import io

import requests
import docx
import json

In [15]:
# PARSING THE DOCUMENT

# def clean_line(line):
#     line = line.strip()
#     line = line.strip('\uFEFF')
#     return line

# def read_faq(file_id):
#     url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    
#     response = requests.get(url)
#     response.raise_for_status()
    
#     with io.BytesIO(response.content) as f_in:
#         doc = docx.Document(f_in)

#     questions = []

#     question_heading_style = 'heading 2'
#     section_heading_style = 'heading 1'
    
#     heading_id = ''
#     section_title = ''
#     question_title = ''
#     answer_text_so_far = ''
     
#     for p in doc.paragraphs:
#         style = p.style.name.lower()
#         p_text = clean_line(p.text)
    
#         if len(p_text) == 0:
#             continue
    
#         if style == section_heading_style:
#             section_title = p_text
#             continue
    
#         if style == question_heading_style:
#             answer_text_so_far = answer_text_so_far.strip()
#             if answer_text_so_far != '' and section_title != '' and question_title != '':
#                 questions.append({
#                     'text': answer_text_so_far,
#                     'section': section_title,
#                     'question': question_title,
#                 })
#                 answer_text_so_far = ''
    
#             question_title = p_text
#             continue
        
#         answer_text_so_far += '\n' + p_text
    
#     answer_text_so_far = answer_text_so_far.strip()
#     if answer_text_so_far != '' and section_title != '' and question_title != '':
#         questions.append({
#             'text': answer_text_so_far,
#             'section': section_title,
#             'question': question_title,
#         })

#     return questions

# faq_documents = {
#     'data-engineering-zoomcamp': '19bnYs80DwuUimHM65UV3sylsCn2j1vziPOwzBwQrebw',
#     'machine-learning-zoomcamp': '1LpPanc33QJJ6BSsyxVg-pWNMplal84TdZtq10naIhD8',
#     'mlops-zoomcamp': '12TlBfhIiKtyBv8RnsoJR6F72bkPDGEvPOItJIxaEzE0',
# }


# documents = []

# for course, file_id in faq_documents.items():
#     print(course)
#     course_documents = read_faq(file_id)
#     documents.append({'course': course, 'documents': course_documents})

# with open('documents.json', 'wt') as f_out:
#     json.dump(documents, f_out, indent=2)

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [35]:

# !head documents.json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# Generating answers with OpenAI

In [29]:
# from openai import OpenAI()
from openai._client import OpenAI

import os
os.environ['OPENAI_API_KEY'] = "---ADD_KEY---"

client = OpenAI()


In [31]:
# turn documents into a prompt

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teacher assistant. Answer the QUESTION based on CONTEXT from the FAQ database. 
    Use only the facts for the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.


    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [32]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )

    return response.choices[0].message.content

# Elastic search

In [8]:
# index documents with Elastic Search and later use them for searching.
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('httlp://localhost:9200')

In [11]:
# create an index // an index in ES is like a table in database

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
from tqdm.auto import tqdm

In [15]:
# tqdm is used for the progess bar
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/1023 [00:00<?, ?it/s]

In [ ]:
user_question = "How do I join the course after it has started?"

In [22]:
def elastic_search(user_question):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": user_question,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [24]:
# elastic_search(user_question)

In [27]:
def rag(query):
    search_results = elastic_search(query)      # replace with the search of choice
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [33]:
rag(user_question)

'Yes, you can still join the course after it has started. Be aware that there will be deadlines for turning in the final projects, so it is advisable not to leave everything for the last minute.'